<a href="https://colab.research.google.com/github/sevincsemih76/01-02-exploring-the-relationship-between-gender-and-policing/blob/master/TEZ_PAGERANK_01_12_ipynb_adl%C4%B1_not_defterinin_kopyas%C4%B1_adl%C4%B1_not_defterinin_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Drive'ı bağlama
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#%%capture
!pip install -U accelerate peft bitsandbytes transformers trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 45.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 43.4 MB/s eta 0:00:00


In [ ]:
!pip install transformers torch accelerate sentencepiece
!pip install sumy nltk rouge-score
!pip install openai
!pip install jsonlines
!pip install scikit-learn
!pip install networkx
!pip install spacy pytextrank

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 137.2 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=f4c6b3da0205f9a986a3f0d6a6d04d1b262c5f6f1072aaf1d3a03bcb8d3ba680
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
  Created wheel for breadability: filename=breadability-0.1.20-py2.py3-none-any.whl size=21695 sha256=b24a2b9790c114fded7b65330223887bd271892dcb6183ad976b2563efb049a2
  Stored in directory: /root/.cache/pip/wheels/32/99/64/59305409cacd03aa03e7bddf31a9db34b1fa7033bd41972662
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=0a5eb27f268b92039388948dedd8ec68d4152682b22687e77acd6b1a0f721f9d
  Stored in directory: /ro

In [ ]:
import json
import jsonlines
import pandas as pd
import numpy as np
from typing import List, Dict, Any
import warnings
import os
warnings.filterwarnings('ignore')

import nltk
nltk.download('punkt_tab')


from google.colab import userdata

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from huggingface_hub import login
import openai
from openai import OpenAI


import spacy
import pytextrank
import nltk
from nltk.corpus import stopwords
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
from collections import Counter


from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize, sent_tokenize

print("Kütüphaneler başarıyla yüklendi!")


print("\nSpacy model yükleniyor...")
try:
    nlp = spacy.load("en_core_web_lg")
except OSError:
    # Model bulunamazsa indir
    print("English model bulunamadı, indiriliyor...")
    !python -m spacy download en_core_web_lg
    nlp = spacy.load("en_core_web_lg")


if "textrank" not in nlp.pipe_names:
    nlp.add_pipe("textrank")

print("Spacy model başarıyla yüklendi!")


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


/usr/local/lib/python3.12/dist-packages
Kütüphaneler başarıyla yüklendi!

Spacy model yükleniyor...
English model bulunamadı, indiriliyor...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 3.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Spacy model başarıyla yüklendi!


In [ ]:
# Google Colab userdata'dan API key'leri al
hf_token = userdata.get('HF_TOKEN_T')
openai_api_key = userdata.get('OPENAI_API_Key')

# Hugging Face token'ını environment variable olarak ayarla
os.environ['HF_TOKEN'] = hf_token
os.environ['HUGGINGFACE_HUB_TOKEN'] = hf_token

# Hugging Face'e giriş
login(token=hf_token)

# OpenAI API'yi ayarla
client = OpenAI(api_key=openai_api_key)

print("Token ayarlandı!")
print("API Key'ler tanımlandı!")

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


Token ayarlandı!
API Key'ler tanımlandı!


In [ ]:
def load_data(file_path: str) -> pd.DataFrame:
    """JSONL dosyasını yükler ve DataFrame'e dönüştürür. Hatalı satırları atlar."""
    data = []
    error_count = 0
    total_lines = 0

    print(f"Veri dosyası yükleniyor: {file_path}")

    with open(file_path, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, start=1):
            total_lines = line_num
            line = line.strip()

            if not line:
                continue

            try:
                obj = json.loads(line)
                data.append(obj)
            except json.JSONDecodeError as e:
                error_count += 1
                if error_count <= 10:
                    print(f"Uyarı: Satır {line_num} atlandı - JSON hatası: {str(e)[:100]}")
                elif error_count == 11:
                    print(f"Uyarı: Daha fazla hatalı satır var, sessizce atlanıyor...")
                continue
            except Exception as e:
                error_count += 1
                if error_count <= 10:
                    print(f"Uyarı: Satır {line_num} atlandı - {str(e)[:100]}")
                continue

    print(f"\n{'='*80}")
    print(f"Veri Yükleme Özeti:")
    print(f"  Toplam satır sayısı: {total_lines}")
    print(f"  Başarıyla yüklenen satır: {len(data)}")
    if error_count > 0:
        print(f"  Atlanan hatalı satır: {error_count}")
    print(f"{'='*80}\n")

    if len(data) == 0:
        raise ValueError("Hiç geçerli veri yüklenemedi! JSONL dosyasını kontrol edin.")

    return pd.DataFrame(data)

def analyze_dataset(df: pd.DataFrame):
    """Veri setinin detaylı analizini yapar."""
    print("=" * 80)
    print("VERİ SETİ ANALİZİ")
    print("=" * 80)

    # Temel bilgiler
    print(f"\n1. Satır Sayısı: {len(df)}")
    print(f"2. Sütun Sayısı: {len(df.columns)}")
    print(f"3. Sütun İsimleri: {list(df.columns)}")

    # Veri tipleri
    print("\n4. Veri Tipleri:")
    print(df.dtypes)

    # Null değer kontrolü
    print("\n5. Null Değer Kontrolü:")
    null_counts = df.isnull().sum()
    for col, count in null_counts.items():
        print(f"   {col}: {count} null değer")

    # İlk 5 satır
    print("\n6. İlk 5 Satır:")
    print(df.head())

    # Soru ve cevap sayıları
    if 'questions' in df.columns and 'answers' in df.columns:
        question_counts = df['questions'].apply(lambda x: len(x) if isinstance(x, list) else 1)
        answer_counts = df['answers'].apply(lambda x: len(x) if isinstance(x, list) else 1)
        print(f"\n7. Ortalama Soru Sayısı: {question_counts.mean():.2f}")
        print(f"8. Ortalama Cevap Sayısı: {answer_counts.mean():.2f}")

    # Story sütunu karakter istatistikleri
    if 'story' in df.columns:
        story_lengths = df['story'].astype(str).apply(len)
        print(f"\n9. Story Sütunu İstatistikleri:")
        print(f"   Ortalama karakter sayısı: {story_lengths.mean():.2f}")
        print(f"   Minimum karakter sayısı: {story_lengths.min()}")
        print(f"   Maksimum karakter sayısı: {story_lengths.max()}")
        print(f"   Medyan karakter sayısı: {story_lengths.median():.2f}")

    # Summary sütunu karakter istatistikleri
    if 'summary' in df.columns:
        summary_lengths = df['summary'].astype(str).apply(len)
        print(f"\n10. Summary Sütunu İstatistikleri:")
        print(f"   Ortalama karakter sayısı: {summary_lengths.mean():.2f}")
        print(f"   Minimum karakter sayısı: {summary_lengths.min()}")
        print(f"   Maksimum karakter sayısı: {summary_lengths.max()}")
        print(f"   Medyan karakter sayısı: {summary_lengths.median():.2f}")

    print("\n" + "=" * 80)

# Veri setini yükleme
data_path = "/content/data.jsonl"
df = load_data(data_path)
analyze_dataset(df)

Veri dosyası yükleniyor: /content/data.jsonl

Veri Yükleme Özeti:
  Toplam satır sayısı: 10388
  Başarıyla yüklenen satır: 10388

VERİ SETİ ANALİZİ

1. Satır Sayısı: 10388
2. Sütun Sayısı: 4
3. Sütun İsimleri: ['story', 'questions', 'answers', 'summary']

4. Veri Tipleri:
story        object
questions    object
answers      object
summary      object
dtype: object

5. Null Değer Kontrolü:
   story: 0 null değer
   questions: 0 null değer
   answers: 0 null değer
   summary: 0 null değer

6. İlk 5 Satır:
                                               story  \
0  'SINDH KALAY', England (CNN) -- The aroma of f...   
1  (AOL Autos)  -- At the 2009 Detroit Auto Show,...   
2  (AOL Autos)  -- Buying a used car may seem lik...   
3  (AOL Autos)  -- Collecting cars is an expensiv...   
4  (AOL Autos)  -- Have you ever wondered what yo...   

                                           questions  \
0  [Where were they being deployed to?, What does...   
1  [What debuts at the 2009 Detroit Auto S

In [ ]:
# print(df['summary'].iloc[0])

Veri Seçimi

In [ ]:
# Veri Seçimi Fonksiyonu
def select_data(df: pd.DataFrame, selection: str) -> pd.DataFrame:
    """
    Veri seçimi yapar.
    Örnekler: "1:10" -> ilk 10 satır, "1:10000" -> ilk 10000 satır, "20:30" -> 20-30. satırlar
    """
    if ":" in selection:
        parts = selection.split(":")
        start = int(parts[0]) - 1 if parts[0] else 0
        end = int(parts[1]) if parts[1] else len(df)
        selected_df = df.iloc[start:end].copy()
    else:
        # Sadece sayı verilirse, ilk N satırı al
        n = int(selection)
        selected_df = df.iloc[:n].copy()

    print(f"Seçilen veri: {len(selected_df)} satır")
    return selected_df.reset_index(drop=True)

# Kullanıcıdan veri seçimi (örnek: "1:10" veya "10" veya "20:30")
# Bu değeri değiştirebilirsiniz:
# - "10" -> ilk 10 satır
# - "1:10" -> ilk 10 satır
# - "20:30" -> 20-30. satırlar
# - "all" veya "tümü" -> tüm satırlar
DATA_SELECTION = "551:580"  # Bu değeri değiştirebilirsiniz

if DATA_SELECTION.lower() in ["all", "tümü", "hepsi"]:
    selected_df = df.copy()
    print(f"Seçilen veri: Tüm satırlar ({len(selected_df)} satır)")
else:
    selected_df = select_data(df, DATA_SELECTION)


Seçilen veri: 30 satır


Model Yüklemeleri

In [ ]:
print("Llama-3.1-8B-Instruct modeli yükleniyor...")
model_name = "meta-llama/Llama-3.1-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, token=hf_token)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=hf_token,
    torch_dtype=torch.float16,
    device_map="auto"
)


llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)

print("Model başarıyla yüklendi!")

Llama-3.1-8B-Instruct modeli yükleniyor...


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Device set to use cuda:0


Model başarıyla yüklendi!


Adım 1 - Llama ile Özet-1 Oluşturma

In [ ]:
def generate_summary_1(text: str, pipeline_obj) -> str:
    """Llama-3.1-8B-Instruct ile özet-1 oluşturur."""
    prompt = """Make an abstract summary.

Text to summarize:
{}

Summary:""".format(text)

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]

    formatted_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.7,
            do_sample=True,
            top_p=0.9
        )

    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return response.strip()

print("\n" + "=" * 80)
print("ADIM 1: ÖZET-1 OLUŞTURULUYOR")
print("=" * 80)

summaries_1 = []
for idx, row in selected_df.iterrows():
    story = row['story']
    summary_1 = generate_summary_1(story, llama_pipeline)
    summaries_1.append(summary_1)
    print(f"\nİlerleme: {idx+1}/{len(selected_df)}")

# Özet kalitesi değerlendirmesi
original_lengths = selected_df['story'].astype(str).apply(len)
summary_1_lengths = [len(s) for s in summaries_1]
empty_summaries = sum(1 for s in summaries_1 if not s.strip())

print("\n" + "=" * 80)
print("Özet Kalitesi Değerlendirmesi:")
print("=" * 80)
print(f"Ortalama orijinal metin uzunluğu: {original_lengths.mean():.2f} karakter")
print(f"Ortalama özet uzunluğu: {np.mean(summary_1_lengths):.2f} karakter")
print(f"Boş özet sayısı: {empty_summaries}")
print("Adım 1 tamamlandı: Özet-1 oluşturuldu!")

# Özet-1 sonuçlarını yazdırma
print("\n" + "=" * 80)
print("ADIM 1 SONUÇLARI - ORİJİNAL METİNLER VE ÖZET-1")
print("=" * 80)
for idx, row in selected_df.iterrows():
    print(f"\n--- Story {idx+1} ---")
    print(f"Orijinal Metin:\n{row['story']}\n")
    print(f"Özet-1:\n{summaries_1[idx]}\n")
    print("-" * 80)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



ADIM 1: ÖZET-1 OLUŞTURULUYOR


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



İlerleme: 1/30


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...


İlerleme: 27/30


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



İlerleme: 28/30


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



İlerleme: 29/30

İlerleme: 30/30

Özet Kalitesi Değerlendirmesi:
Ortalama orijinal metin uzunluğu: 2642.50 karakter
Ortalama özet uzunluğu: 484.17 karakter
Boş özet sayısı: 0
Adım 1 tamamlandı: Özet-1 oluşturuldu!

ADIM 1 SONUÇLARI - ORİJİNAL METİNLER VE ÖZET-1

--- Story 1 ---
Orijinal Metin:
(CNN)  -- There was no mistaking the target: the eight huge cooling towers at Ratcliffe-on-Soar power station, sending plumes of steam high into the watery blue sky of the English Midlands. A mix of peaceful protest and direct action took place at Ratcliffe-on-Soar, England. Instead the question in the minds of an estimated 1,000 protestors gathered in the surrounding woods and scrubland was how could they get in and shut it down. Surrounded by electrified fences, coils of razor wire and hundreds of police, this coal-fired power station run by German energy firm E.On was the target of environmental activists campaigning to stop climate change. Organizers of the protest, an amorphous group called

Adım 2 - PageRank ile Anahtar Kelime Çıkarma

In [ ]:
def extract_keywords_pagerank(text, num_keywords=8):
    """Extract keywords from text using PageRank algorithm."""
    import networkx as nx
    from collections import defaultdict

    # Metni tokenize et ve temizle
    doc = nlp(text)

    # Stopwords'leri filtrele
    stop_words = set(stopwords.words('english'))
    words = [token.text.lower() for token in doc if token.is_alpha and not token.is_stop and len(token.text) > 2]

    if len(words) < 2:
        return []

    # Graf oluştur
    G = nx.Graph()

    # Kelimeler arasında edge'ler oluştur (sliding window yaklaşımı)
    window_size = 3
    for i in range(len(words) - window_size + 1):
        window = words[i:i+window_size]
        for j in range(len(window)):
            for k in range(j+1, len(window)):
                if window[j] != window[k]:
                    if G.has_edge(window[j], window[k]):
                        G[window[j]][window[k]]['weight'] += 1
                    else:
                        G.add_edge(window[j], window[k], weight=1)

    if len(G.nodes()) == 0:
        return []

    # PageRank hesapla
    try:
        pagerank_scores = nx.pagerank(G, max_iter=100)
    except:
        # Eğer PageRank başarısız olursa, basit bir skorlama yap
        pagerank_scores = {node: 1.0 for node in G.nodes()}

    # Skorlara göre sırala
    sorted_words = sorted(pagerank_scores.items(), key=lambda x: x[1], reverse=True)

    # En yüksek skorlu kelimeleri al
    keywords = [word for word, score in sorted_words[:num_keywords]]

    return keywords

print("\n" + "=" * 80)
print("ADIM 2: PAGERANK İLE ANAHTAR KELİME ÇIKARMA")
print("=" * 80)

all_keywords = []
for idx, row in selected_df.iterrows():
    story = str(row['story'])  # String'e çevir
    keywords = extract_keywords_pagerank(story)
    all_keywords.append(keywords)

    if idx < 3:
        print(f"\nStory {idx+1} Debug:")
        print(f"  Metin uzunluğu: {len(story)} karakter")
        print(f"  Çıkarılan anahtar kelimeler: {keywords}")
    else:
        print(f"\nİlerleme: {idx+1}/{len(selected_df)}")

print("\nPAGERANK sonuçları:")
print("=" * 80)
for idx, keywords in enumerate(all_keywords):
    print(f"Story {idx+1}: {keywords}")


ADIM 2: PAGERANK İLE ANAHTAR KELİME ÇIKARMA

Story 1 Debug:
  Metin uzunluğu: 4111 karakter
  Çıkarılan anahtar kelimeler: ['protest', 'climate', 'action', 'police', 'station', 'coal', 'power', 'activists']

Story 2 Debug:
  Metin uzunluğu: 3818 karakter
  Çıkarılan anahtar kelimeler: ['film', 'films', 'foreign', 'best', 'like', 'picture', 'academy', 'said']

Story 3 Debug:
  Metin uzunluğu: 2716 karakter
  Çıkarılan anahtar kelimeler: ['blatter', 'ireland', 'fifa', 'world', 'committee', 'said', 'finals', 'second']

İlerleme: 4/30

İlerleme: 5/30

İlerleme: 6/30

İlerleme: 7/30

İlerleme: 8/30

İlerleme: 9/30

İlerleme: 10/30

İlerleme: 11/30

İlerleme: 12/30

İlerleme: 13/30

İlerleme: 14/30

İlerleme: 15/30

İlerleme: 16/30

İlerleme: 17/30

İlerleme: 18/30

İlerleme: 19/30

İlerleme: 20/30

İlerleme: 21/30

İlerleme: 22/30

İlerleme: 23/30

İlerleme: 24/30

İlerleme: 25/30

İlerleme: 26/30

İlerleme: 27/30

İlerleme: 28/30

İlerleme: 29/30

İlerleme: 30/30

PAGERANK sonuçları:
Stor

Adım 3 - PageRank + Soru-Cevaplar ile Özet-2 Oluşturma

In [ ]:
def generate_summary_2(text: str, keywords: List[str], questions: List[str], answers: List[str], pipeline_obj) -> str:
    """PageRank anahtar kelimeleri ve soru-cevapları kullanarak özet-2 oluşturur."""

    # Soru ve cevapları formatlama
    qa_text = ""
    if isinstance(questions, list) and isinstance(answers, list):
        for q, a in zip(questions, answers):
            qa_text += f"Q: {q}\nA: {a}\n\n"
    elif isinstance(questions, str) and isinstance(answers, str):
        qa_text = f"Q: {questions}\nA: {answers}\n\n"

    keywords_str = ", ".join(keywords)

    prompt = prompt = """You are an expert text summarization assistant. Your SOLE goal is to produce a very CONCISE abstract summary.

**CRITICAL CONSTRAINT: The summary MUST be strictly between 90 and 350 characters (2-3 sentences long).**

Your task is to create a concise, accurate, and comprehensive summary of the text below, incorporating the core concepts from the provided Keywords and Questions & Answers, while adhering to the word count limit. The final summary must address the core information contained in the Questions and Answers.

Keywords: {}

Questions and Answers:
{}

Text to summarize:
{}

Summary: **REMINDER: The summary must NOT exceed 400 characters and MUST include the Q/A information.**""".format(keywords_str, qa_text, text)

    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
    ]

    formatted_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=120,
            temperature=0.7,
            do_sample=True,
            top_p=0.9
        )

    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return response.strip()

print("\n" + "=" * 80)
print("ADIM 3: ÖZET-2 OLUŞTURULUYOR")
print("=" * 80)

summaries_2 = []
for idx, row in selected_df.iterrows():
    story = row['story']
    keywords = all_keywords[idx]
    questions = row['questions']
    answers = row['answers']
    summary_2 = generate_summary_2(story, keywords, questions, answers, llama_pipeline)
    summaries_2.append(summary_2)
    print(f"\nİlerleme: {idx+1}/{len(selected_df)}")

# Özet kalitesi değerlendirmesi
summary_2_lengths = [len(s) for s in summaries_2]
empty_summaries_2 = sum(1 for s in summaries_2 if not s.strip())

print("\n" + "=" * 80)
print("Özet Kalitesi Değerlendirmesi:")
print("=" * 80)
print(f"Ortalama orijinal metin uzunluğu: {original_lengths.mean():.2f} karakter")
print(f"Ortalama özet uzunluğu: {np.mean(summary_2_lengths):.2f} karakter")
print(f"Boş özet sayısı: {empty_summaries_2}")
print("Adım 3 tamamlandı: Özet-2 oluşturuldu!")

# Özet-2 sonuçlarını yazdırma
print("\n" + "=" * 80)
print("ADIM 3 SONUÇLARI - ORİJİNAL METİNLER VE ÖZET-2")
print("=" * 80)
for idx, row in selected_df.iterrows():
    print(f"\n--- Story {idx+1} ---")
    print(f"Orijinal Metin:\n{row['story']}\n")
    print(f"Özet-2:\n{summaries_2[idx]}\n")
    print("-" * 80)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



ADIM 3: ÖZET-2 OLUŞTURULUYOR


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



İlerleme: 1/30


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...


İlerleme: 27/30


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



İlerleme: 28/30


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



İlerleme: 29/30

İlerleme: 30/30

Özet Kalitesi Değerlendirmesi:
Ortalama orijinal metin uzunluğu: 2642.50 karakter
Ortalama özet uzunluğu: 267.20 karakter
Boş özet sayısı: 0
Adım 3 tamamlandı: Özet-2 oluşturuldu!

ADIM 3 SONUÇLARI - ORİJİNAL METİNLER VE ÖZET-2

--- Story 1 ---
Orijinal Metin:
(CNN)  -- There was no mistaking the target: the eight huge cooling towers at Ratcliffe-on-Soar power station, sending plumes of steam high into the watery blue sky of the English Midlands. A mix of peaceful protest and direct action took place at Ratcliffe-on-Soar, England. Instead the question in the minds of an estimated 1,000 protestors gathered in the surrounding woods and scrubland was how could they get in and shut it down. Surrounded by electrified fences, coils of razor wire and hundreds of police, this coal-fired power station run by German energy firm E.On was the target of environmental activists campaigning to stop climate change. Organizers of the protest, an amorphous group called

Adım 4 - GPT-4o ile Özet Değerlendirme ve Puanlama

In [ ]:
def evaluate_summaries_gpt4o(original_text: str, summary_1: str, summary_2: str,
                             questions: List[str], answers: List[str]) -> Dict[str, Any]:
    """GPT-4o ile özetleri değerlendirir ve puanlar."""

    # Soru ve cevapları formatlama
    qa_text = ""
    if isinstance(questions, list) and isinstance(answers, list):
        for q, a in zip(questions, answers):
            qa_text += f"Q: {q}\nA: {a}\n\n"
    elif isinstance(questions, str) and isinstance(answers, str):
        qa_text = f"Q: {questions}\nA: {answers}\n\n"

    evaluation_prompt = f"""You are an expert evaluator of text summaries. Your task is to evaluate two summaries (Summary-1 and Summary-2) based on the original text and the questions-answers provided.

Original Text:
{original_text}

Questions and Answers:
{qa_text}

Summary-1:
{summary_1}

Summary-2:
{summary_2}

Please evaluate both summaries based on the following criteria (each out of 10 points):

1. Comprehensiveness: Does the summary cover all main ideas and key points?
2. Accuracy: Is the summary factually correct and faithful to the original text?
3. Relevance: Does the summary address the questions and provide appropriate answers?
4. Coherence: Is the summary well-structured and logically organized?
5. Clarity: Is the summary clear and easy to understand?
6. Conciseness: Is the summary appropriately concise (3-5 sentences)?
7. Completeness: Does the summary answer the questions from the dataset?
8. Information Quality: Is the information presented accurately and usefully?
9. Language Quality: Is the language correct and fluent?
10. Overall Quality: Overall assessment of the summary quality

Please provide your evaluation in the following JSON format:
{{
    "summary_1": {{
        "comprehensiveness": <score>,
        "accuracy": <score>,
        "relevance": <score>,
        "coherence": <score>,
        "clarity": <score>,
        "conciseness": <score>,
        "completeness": <score>,
        "information_quality": <score>,
        "language_quality": <score>,
        "overall_quality": <score>
    }},
    "summary_2": {{
        "comprehensiveness": <score>,
        "accuracy": <score>,
        "relevance": <score>,
        "coherence": <score>,
        "clarity": <score>,
        "conciseness": <score>,
        "completeness": <score>,
        "information_quality": <score>,
        "language_quality": <score>,
        "overall_quality": <score>
    }}
}}
"""

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an expert evaluator. Provide evaluations in JSON format only."},
                {"role": "user", "content": evaluation_prompt}
            ],
            temperature=0.3,
            max_tokens=2000
        )

        result_text = response.choices[0].message.content

        import re
        json_match = re.search(r'\{.*\}', result_text, re.DOTALL)
        if json_match:
            result_json = json.loads(json_match.group())
            return result_json
        else:
            return {"error": "Could not parse JSON"}
    except Exception as e:
        return {"error": str(e)}

print("\n" + "=" * 80)
print("ADIM 4: GPT-4o İLE ÖZET DEĞERLENDİRME VE PUANLAMA")
print("=" * 80)

evaluations = []
for idx, row in selected_df.iterrows():
    print(f"\nDeğerlendiriliyor: {idx+1}/{len(selected_df)}")
    eval_result = evaluate_summaries_gpt4o(
        row['story'],
        summaries_1[idx],
        summaries_2[idx],
        row['questions'],
        row['answers']
    )
    evaluations.append(eval_result)

# Değerlendirme sonuçlarını yazdırma
print("\n" + "=" * 80)
print("ADIM 4 SONUÇLARI - GPT-4o DEĞERLENDİRMELERİ")
print("=" * 80)

for idx, eval_result in enumerate(evaluations):
    print(f"\n--- Story {idx+1} Değerlendirmesi ---")
    if "error" not in eval_result:
        if "summary_1" in eval_result:
            print("\nÖZET-1 PUANLARI:")
            for criterion, score in eval_result["summary_1"].items():
                print(f"  {criterion}: {score}/10")
            avg_1 = sum(eval_result["summary_1"].values()) / len(eval_result["summary_1"])
            print(f"  Ortalama: {avg_1:.2f}/10")

        if "summary_2" in eval_result:
            print("\nÖZET-2 PUANLARI:")
            for criterion, score in eval_result["summary_2"].items():
                print(f"  {criterion}: {score}/10")
            avg_2 = sum(eval_result["summary_2"].values()) / len(eval_result["summary_2"])
            print(f"  Ortalama: {avg_2:.2f}/10")
    else:
        print(f"  Hata: {eval_result['error']}")
    print("-" * 80)


ADIM 4: GPT-4o İLE ÖZET DEĞERLENDİRME VE PUANLAMA

Değerlendiriliyor: 1/30

Değerlendiriliyor: 2/30

Değerlendiriliyor: 3/30

Değerlendiriliyor: 4/30

Değerlendiriliyor: 5/30

Değerlendiriliyor: 6/30

Değerlendiriliyor: 7/30

Değerlendiriliyor: 8/30

Değerlendiriliyor: 9/30

Değerlendiriliyor: 10/30

Değerlendiriliyor: 11/30

Değerlendiriliyor: 12/30

Değerlendiriliyor: 13/30

Değerlendiriliyor: 14/30

Değerlendiriliyor: 15/30

Değerlendiriliyor: 16/30

Değerlendiriliyor: 17/30

Değerlendiriliyor: 18/30

Değerlendiriliyor: 19/30

Değerlendiriliyor: 20/30

Değerlendiriliyor: 21/30

Değerlendiriliyor: 22/30

Değerlendiriliyor: 23/30

Değerlendiriliyor: 24/30

Değerlendiriliyor: 25/30

Değerlendiriliyor: 26/30

Değerlendiriliyor: 27/30

Değerlendiriliyor: 28/30

Değerlendiriliyor: 29/30

Değerlendiriliyor: 30/30

ADIM 4 SONUÇLARI - GPT-4o DEĞERLENDİRMELERİ

--- Story 1 Değerlendirmesi ---

ÖZET-1 PUANLARI:
  comprehensiveness: 8/10
  accuracy: 9/10
  relevance: 9/10
  coherence: 8/10
  c

Adım 5 - Metrik Değerlendirmeleri (BLEU, ROUGE, G-Eval)

In [ ]:
def calculate_bleu(reference: str, candidate: str) -> float:
    """BLEU skorunu hesaplar."""
    try:
        # String'e çevirme ve temizleme
        reference = str(reference).strip() if reference is not None else ""
        candidate = str(candidate).strip() if candidate is not None else ""

        if not reference or not candidate or len(reference) == 0 or len(candidate) == 0:
            return 0.0

        # Tokenize etme
        ref_tokens = word_tokenize(reference.lower())
        cand_tokens = word_tokenize(candidate.lower())

        # Boş token kontrolü
        if len(ref_tokens) == 0 or len(cand_tokens) == 0:
            return 0.0

        smoothing = SmoothingFunction().method4

        bleu_score = sentence_bleu([ref_tokens], cand_tokens, smoothing_function=smoothing)

        # BLEU skoru 0-1 arasında olmalı, eğer NaN veya negatif ise 0 döndür
        if bleu_score is None or bleu_score < 0:
            return 0.0

        # NaN kontrolü
        try:
            if np.isnan(bleu_score) or np.isinf(bleu_score):
                return 0.0
        except:
            pass

        return float(bleu_score)
    except Exception as e:
        # Hata durumunda 0 döndür
        return 0.0

def calculate_rouge(reference: str, candidate: str) -> Dict[str, float]:
    """ROUGE skorlarını hesaplar."""
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference, candidate)
    return {
        'rouge1': scores['rouge1'].fmeasure,
        'rouge2': scores['rouge2'].fmeasure,
        'rougeL': scores['rougeL'].fmeasure
    }

def calculate_geval(reference: str, candidate: str) -> float:
    """G-Eval benzeri bir skor hesaplar (basitleştirilmiş versiyon)."""
    try:
        # String'e çevirme ve temizleme
        reference = str(reference).strip() if reference is not None else ""
        candidate = str(candidate).strip() if candidate is not None else ""

        # Boş string kontrolü
        if not reference or not candidate or len(reference) == 0 or len(candidate) == 0:
            return 0.0

        # Stopwords'leri filtreleme
        stop_words = set(stopwords.words('english'))

        # Tokenize et ve filtreleme
        ref_tokens = [w for w in word_tokenize(reference.lower()) if w.isalnum() and w not in stop_words]
        cand_tokens = [w for w in word_tokenize(candidate.lower()) if w.isalnum() and w not in stop_words]

        # Boş token kontrolü
        if len(ref_tokens) == 0 or len(cand_tokens) == 0:
            return 0.0

        # Jaccard benzerliği ve token overlap
        ref_set = set(ref_tokens)
        cand_set = set(cand_tokens)

        overlap = len(ref_set.intersection(cand_set))
        union = len(ref_set.union(cand_set))

        if union == 0:
            return 0.0

        # Jaccard benzerliği
        jaccard = overlap / union

        # Token overlap oranı (precision benzeri)
        token_overlap = overlap / len(ref_set) if len(ref_set) > 0 else 0.0

        # İkisinin ortalaması
        geval_score = (jaccard + token_overlap) / 2.0

        # NaN ve inf kontrolü
        if isinstance(geval_score, float) and (np.isnan(geval_score) or np.isinf(geval_score)):
            return 0.0

        return float(geval_score)
    except Exception as e:
        # Hata durumunda 0 döndür
        return 0.0

print("\n" + "=" * 80)
print("ADIM 5: METRİK DEĞERLENDİRMELERİ (BLEU, ROUGE, G-Eval)")
print("=" * 80)

# Referans özetler (veri setindeki summary sütunu)
print("\nReferans özetler kontrol ediliyor...")
print(f"Summary sütunu var mı: {'summary' in selected_df.columns}")
if 'summary' in selected_df.columns:
    print(f"Summary sütunu tipi: {selected_df['summary'].dtype}")
    print(f"İlk summary örneği: {str(selected_df['summary'].iloc[0])[:100] if len(selected_df) > 0 else 'YOK'}")
    print(f"Boş summary sayısı: {selected_df['summary'].isna().sum()}")

reference_summaries = selected_df['summary'].astype(str).tolist()
print(f"Toplam {len(reference_summaries)} referans özet yüklendi")

# Metrikleri hesaplama
bleu_scores_1 = []
bleu_scores_2 = []
rouge_scores_1 = []
rouge_scores_2 = []
geval_scores_1 = []
geval_scores_2 = []

for idx in range(len(selected_df)):
    ref = str(reference_summaries[idx]).strip() if reference_summaries[idx] is not None else ""
    sum1 = str(summaries_1[idx]).strip() if summaries_1[idx] is not None else ""
    sum2 = str(summaries_2[idx]).strip() if summaries_2[idx] is not None else ""

    # Debug: İlk birkaç örnek için bilgi göster
    if idx < 2:
        print(f"\nDebug - Story {idx+1}:")
        print(f"  Reference uzunluğu: {len(ref)} karakter")
        print(f"  Reference örneği: {ref[:100] if len(ref) > 0 else 'BOŞ'}...")
        print(f"  Summary-1 uzunluğu: {len(sum1)} karakter")
        print(f"  Summary-1 örneği: {sum1[:100] if len(sum1) > 0 else 'BOŞ'}...")
        print(f"  Summary-2 uzunluğu: {len(sum2)} karakter")
        print(f"  Summary-2 örneği: {sum2[:100] if len(sum2) > 0 else 'BOŞ'}...")

    # Özet-1 metrikleri
    bleu_1 = calculate_bleu(ref, sum1)
    rouge_1 = calculate_rouge(ref, sum1)
    geval_1 = calculate_geval(ref, sum1)

    bleu_scores_1.append(bleu_1)
    rouge_scores_1.append(rouge_1)
    geval_scores_1.append(geval_1)

    # Özet-2 metrikleri
    bleu_2 = calculate_bleu(ref, sum2)
    rouge_2 = calculate_rouge(ref, sum2)
    geval_2 = calculate_geval(ref, sum2)

    bleu_scores_2.append(bleu_2)
    rouge_scores_2.append(rouge_2)
    geval_scores_2.append(geval_2)

    # Debug: İlk birkaç örnek için skorları gösterme
    if idx < 2:
        print(f"  BLEU-1: {bleu_1:.4f}, G-Eval-1: {geval_1:.4f}")
        print(f"  BLEU-2: {bleu_2:.4f}, G-Eval-2: {geval_2:.4f}")

        # Test: Tokenize kontrolü
        if len(ref) > 0 and len(sum1) > 0:
            ref_tokens = word_tokenize(ref.lower())
            sum1_tokens = word_tokenize(sum1.lower())
            print(f"  Reference token sayısı: {len(ref_tokens)}")
            print(f"  Summary-1 token sayısı: {len(sum1_tokens)}")
            print(f"  Ortak token sayısı: {len(set(ref_tokens) & set(sum1_tokens))}")

# Ortalama skorları hesaplama
avg_bleu_1 = np.mean(bleu_scores_1)
avg_bleu_2 = np.mean(bleu_scores_2)
avg_rouge1_1 = np.mean([r['rouge1'] for r in rouge_scores_1])
avg_rouge1_2 = np.mean([r['rouge1'] for r in rouge_scores_2])
avg_rouge2_1 = np.mean([r['rouge2'] for r in rouge_scores_1])
avg_rouge2_2 = np.mean([r['rouge2'] for r in rouge_scores_2])
avg_rougeL_1 = np.mean([r['rougeL'] for r in rouge_scores_1])
avg_rougeL_2 = np.mean([r['rougeL'] for r in rouge_scores_2])
avg_geval_1 = np.mean(geval_scores_1)
avg_geval_2 = np.mean(geval_scores_2)

# Sonuçları yazdırma
print("\n" + "=" * 80)
print("METRİK SONUÇLARI")
print("=" * 80)
print(f"\n{'Metrik':<20} {'Özet-1':<15} {'Özet-2':<15}")
print("-" * 50)
print(f"{'BLEU':<20} {avg_bleu_1:<15.4f} {avg_bleu_2:<15.4f}")
print(f"{'ROUGE-1':<20} {avg_rouge1_1:<15.4f} {avg_rouge1_2:<15.4f}")
print(f"{'ROUGE-2':<20} {avg_rouge2_1:<15.4f} {avg_rouge2_2:<15.4f}")
print(f"{'ROUGE-L':<20} {avg_rougeL_1:<15.4f} {avg_rougeL_2:<15.4f}")
print(f"{'G-Eval':<20} {avg_geval_1:<15.4f} {avg_geval_2:<15.4f}")

# Detaylı sonuçlar
print("\n" + "=" * 80)
print("DETAYLI METRİK SONUÇLARI (Her Story İçin)")
print("=" * 80)

for idx in range(len(selected_df)):
    print(f"\n--- Story {idx+1} ---")
    print(f"Özet-1 Metrikleri:")
    print(f"  BLEU: {bleu_scores_1[idx]:.4f}")
    print(f"  ROUGE-1: {rouge_scores_1[idx]['rouge1']:.4f}")
    print(f"  ROUGE-2: {rouge_scores_1[idx]['rouge2']:.4f}")
    print(f"  ROUGE-L: {rouge_scores_1[idx]['rougeL']:.4f}")
    print(f"  G-Eval: {geval_scores_1[idx]:.4f}")

    print(f"\nÖzet-2 Metrikleri:")
    print(f"  BLEU: {bleu_scores_2[idx]:.4f}")
    print(f"  ROUGE-1: {rouge_scores_2[idx]['rouge1']:.4f}")
    print(f"  ROUGE-2: {rouge_scores_2[idx]['rouge2']:.4f}")
    print(f"  ROUGE-L: {rouge_scores_2[idx]['rougeL']:.4f}")
    print(f"  G-Eval: {geval_scores_2[idx]:.4f}")
    print("-" * 80)


ADIM 5: METRİK DEĞERLENDİRMELERİ (BLEU, ROUGE, G-Eval)

Referans özetler kontrol ediliyor...
Summary sütunu var mı: True
Summary sütunu tipi: object
İlk summary örneği: Eco activists gathered at power station in UK to protest against climate change .
Called "The Great 
Boş summary sayısı: 0
Toplam 30 referans özet yüklendi

Debug - Story 1:
  Reference uzunluğu: 323 karakter
  Reference örneği: Eco activists gathered at power station in UK to protest against climate change .
Called "The Great ...
  Summary-1 uzunluğu: 487 karakter
  Summary-1 örneği: A group of approximately 1,000 environmental activists, part of the "Camp for Climate Action," stage...
  Summary-2 uzunluğu: 281 karakter
  Summary-2 örneği: Approximately 1,000 eco-activists protested against "The Great Climate Swoop" at Ratcliffe-on-Soar p...
  BLEU-1: 0.0781, G-Eval-1: 0.2885
  BLEU-2: 0.1266, G-Eval-2: 0.3239
  Reference token sayısı: 56
  Summary-1 token sayısı: 88
  Ortak token sayısı: 22

Debug - Story 2:
  Refere

 Genel Sonuçlar ve Özet

In [ ]:
print("\n" + "=" * 80)
print("GENEL SONUÇLAR VE ÖZET")
print("=" * 80)

# GPT-4o puanlarının ortalamaları
if evaluations and "error" not in evaluations[0]:
    gpt4o_scores_1 = []
    gpt4o_scores_2 = []

    for eval_result in evaluations:
        if "summary_1" in eval_result:
            avg_1 = sum(eval_result["summary_1"].values()) / len(eval_result["summary_1"])
            gpt4o_scores_1.append(avg_1)
        if "summary_2" in eval_result:
            avg_2 = sum(eval_result["summary_2"].values()) / len(eval_result["summary_2"])
            gpt4o_scores_2.append(avg_2)

    if gpt4o_scores_1 and gpt4o_scores_2:
        avg_gpt4o_1 = np.mean(gpt4o_scores_1)
        avg_gpt4o_2 = np.mean(gpt4o_scores_2)

        print(f"\nGPT-4o Ortalama Puanları:")
        print(f"  Özet-1: {avg_gpt4o_1:.2f}/10")
        print(f"  Özet-2: {avg_gpt4o_2:.2f}/10")

print(f"\nMetrik Ortalamaları:")
print(f"  Özet-1 BLEU: {avg_bleu_1:.4f}")
print(f"  Özet-2 BLEU: {avg_bleu_2:.4f}")
print(f"  Özet-1 ROUGE-1: {avg_rouge1_1:.4f}")
print(f"  Özet-2 ROUGE-1: {avg_rouge1_2:.4f}")
print(f"  Özet-1 ROUGE-2: {avg_rouge2_1:.4f}")
print(f"  Özet-2 ROUGE-2: {avg_rouge2_2:.4f}")
print(f"  Özet-1 ROUGE-L: {avg_rougeL_1:.4f}")
print(f"  Özet-2 ROUGE-L: {avg_rougeL_2:.4f}")
print(f"  Özet-1 G-Eval: {avg_geval_1:.4f}")
print(f"  Özet-2 G-Eval: {avg_geval_2:.4f}")

# En iyi özeti belirleme
print("\n" + "=" * 80)
print("SONUÇ: EN İYİ ÖZET")
print("=" * 80)

# Metrik skorlarını normalize etme ve karşılaştırma
total_score_1 = avg_bleu_1 + avg_rouge1_1 + avg_rouge2_1 + avg_rougeL_1 + avg_geval_1
total_score_2 = avg_bleu_2 + avg_rouge1_2 + avg_rouge2_2 + avg_rougeL_2 + avg_geval_2

if total_score_1 > total_score_2:
    print("Özet-1 daha yüksek toplam metrik skoruna sahip!")
    print(f"Özet-1 Toplam Skor: {total_score_1:.4f}")
    print(f"Özet-2 Toplam Skor: {total_score_2:.4f}")
elif total_score_2 > total_score_1:
    print("Özet-2 daha yüksek toplam metrik skoruna sahip!")
    print(f"Özet-1 Toplam Skor: {total_score_1:.4f}")
    print(f"Özet-2 Toplam Skor: {total_score_2:.4f}")
else:
    print("Her iki özet de benzer skorlara sahip!")
    print(f"Özet-1 Toplam Skor: {total_score_1:.4f}")
    print(f"Özet-2 Toplam Skor: {total_score_2:.4f}")

# Ortalama karakter sayıları
avg_char_summary_1 = np.mean([len(s) for s in summaries_1])
avg_char_summary_2 = np.mean([len(s) for s in summaries_2])

print("\n" + "=" * 80)
print("ORTALAMA KARAKTER SAYILARI")
print("=" * 80)
print(f"1. Özet (Özet-1) Ortalama Karakter Sayısı: {avg_char_summary_1:.2f}")
print(f"2. Özet (Özet-2) Ortalama Karakter Sayısı: {avg_char_summary_2:.2f}")

print("\n" + "=" * 80)
print("TÜM ADIMLAR TAMAMLANDI!")
print("=" * 80)




GENEL SONUÇLAR VE ÖZET

GPT-4o Ortalama Puanları:
  Özet-1: 8.43/10
  Özet-2: 8.03/10

Metrik Ortalamaları:
  Özet-1 BLEU: 0.0765
  Özet-2 BLEU: 0.1221
  Özet-1 ROUGE-1: 0.3426
  Özet-2 ROUGE-1: 0.4503
  Özet-1 ROUGE-2: 0.1287
  Özet-2 ROUGE-2: 0.2017
  Özet-1 ROUGE-L: 0.2226
  Özet-2 ROUGE-L: 0.2863
  Özet-1 G-Eval: 0.2791
  Özet-2 G-Eval: 0.3643

SONUÇ: EN İYİ ÖZET
Özet-2 daha yüksek toplam metrik skoruna sahip!
Özet-1 Toplam Skor: 1.0495
Özet-2 Toplam Skor: 1.4247

ORTALAMA KARAKTER SAYILARI
1. Özet (Özet-1) Ortalama Karakter Sayısı: 484.17
2. Özet (Özet-2) Ortalama Karakter Sayısı: 267.20

TÜM ADIMLAR TAMAMLANDI!


Özet Görüntüleme


In [ ]:
def display_summaries(selection: str):
    """
    Seçilen özet(ler)i görüntüler.

    Parametreler:
    - selection: "5" -> 5. özeti göster
                 "1:5" -> 1-5. özetleri göster
                 "1,3,5" -> 1., 3. ve 5. özetleri göster
    """
    # Seçimleri parse et
    indices = []

    if ":" in selection:
        # Aralık seçimi (örn: "1:5")
        parts = selection.split(":")
        start = int(parts[0]) - 1 if parts[0] else 0
        end = int(parts[1]) if parts[1] else len(selected_df)
        indices = list(range(start, min(end, len(selected_df))))
    elif "," in selection:
        # Liste seçimi (örn: "1,3,5")
        indices = [int(x.strip()) - 1 for x in selection.split(",") if x.strip().isdigit()]
        indices = [i for i in indices if 0 <= i < len(selected_df)]
    else:
        # Tek özet seçimi (örn: "5")
        idx = int(selection) - 1
        if 0 <= idx < len(selected_df):
            indices = [idx]

    if not indices:
        print("Geçersiz seçim! Lütfen geçerli bir özet numarası veya aralığı girin.")
        return

    # Seçilen özetleri görüntüle
    print("\n" + "=" * 100)
    print("ÖZET GÖRÜNTÜLEME")
    print("=" * 100)

    for idx in indices:
        if idx >= len(selected_df) or idx < 0:
            continue

        row = selected_df.iloc[idx]

        print(f"\n{'='*100}")
        print(f"ÖZET {idx+1}")
        print(f"{'='*100}\n")

        print(f"{'─'*100}")
        print("1. ÖZET (Özet-1):")
        print(f"{'─'*100}")
        print(summaries_1[idx])
        print()

        print(f"{'─'*100}")
        print("2. ÖZET (Özet-2):")
        print(f"{'─'*100}")
        print(summaries_2[idx])
        print()

        print(f"{'─'*100}")
        print("VERİ SETİNDEKİ SUMMARY SÜTUNU:")
        print(f"{'─'*100}")
        print(row['summary'])
        print()

        print(f"{'─'*100}")
        print("VERİ SETİNDEKİ STORY SÜTUNU:")
        print(f"{'─'*100}")
        print(row['story'])
        print()

        print(f"{'='*100}\n")

# Kullanım örnekleri:
# Tek bir özet görüntülemek için:
# display_summaries("5")

# Aralık görüntülemek için:
# display_summaries("1:5")

# Belirli özetleri görüntülemek için:
# display_summaries("1,3,5")

# İstediğiniz özet(ler)i görüntülemek için aşağıdaki satırı düzenleyin:
# Örnek: "5" -> 5. özeti göster, "1:5" -> 1-5. özetleri göster, "1,3,5" -> 1., 3. ve 5. özetleri göster
SUMMARY_SELECTION = "26"  # Bu değeri değiştirebilirsiniz

display_summaries(SUMMARY_SELECTION)



ÖZET GÖRÜNTÜLEME

ÖZET 26

────────────────────────────────────────────────────────────────────────────────────────────────────
1. ÖZET (Özet-1):
────────────────────────────────────────────────────────────────────────────────────────────────────
A truck carrying fireworks for the Fourth of July show on Ocracoke Island in North Carolina exploded, resulting in the deaths of two people and critical injuries to three others, including two volunteer firefighters. The blast occurred at the island docks, and emergency responders were quickly deployed to the scene. The incident is currently under investigation by the FBI and the US Bureau of Alcohol, Tobacco, Firearms and Explosives.

────────────────────────────────────────────────────────────────────────────────────────────────────
2. ÖZET (Özet-2):
────────────────────────────────────────────────────────────────────────────────────────────────────
Two people were killed when a truck carrying fireworks exploded at Ocracoke Island docks. Th